Kodamanchili Sai Pavan Kumar
MTech CSE
213010104

**Importing the colab drive**

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Load text into the memory**

In [ ]:
# load doc into memory 
def load_doc(filename): 
# open the file as read only 
 file = open(filename, 'r') 
# read all text 
 text = file.read() 
# close the file 
 file.close() 
 return text



**Reading the file**

**The dataset that we have taken is a very long text file named as war and peace which congtains various chapters**

In [ ]:
raw_text = load_doc("/content/drive/My Drive/war_and_peace.txt") 
print(raw_text)


CHAPTER I

“Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don’t tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by that
Antichrist—I really believe he is Antichrist—I will have nothing
more to do with you and you are no longer my friend, no longer my
‘faithful slave,’ as you call yourself! But how do you do? I see I
have frightened you—sit down and tell me all the news.”

It was in July, 1805, and the speaker was the well-known Anna Pávlovna
Schérer, maid of honor and favorite of the Empress Márya Fëdorovna.
With these words she greeted Prince Vasíli Kurágin, a man of high
rank and importance, who was the first to arrive at her reception. Anna
Pávlovna had had a cough for some days. She was, as she said, suffering
from la grippe; grippe being then a new word in St. Petersburg, used
only by the elite.

All her invitations without exception, written in French, and delivered
by a scarlet

**Text cleaning**

**Removing whitespaces**

In [ ]:
tokens = raw_text.split() 
raw_text = ' '.join(tokens)


**Creating Sequences**

In [ ]:
# organize into sequences of characters 
length = 10 
sequences = list() 
for i in range(length, len(raw_text)): 
# select sequence of tokens 
 seq = raw_text[i-length:i+1]
 sequences.append(seq) 
print('Total Sequences: %d' % len(sequences)) 



Total Sequences: 270179


**Save the sequences**

In [ ]:
def save_doc(lines, filename): 
 data = '\n'.join(lines) 
 file = open(filename, 'w') 
 file.write(data) 
 file.close()


**Assign filename to prepared sequences**

In [ ]:
# save sequences to file 
out_filename = 'char_sequences.txt' 
save_doc(sequences, out_filename)




**NOW TRAINING USING LSTM**

**We will train our character-based neural language model**

**Here, we now develop a neural language model for the prepared sequence data. This model will read encoded characters and predict the next character in the sequence** 


**Importing all the required libraries**

In [ ]:
from numpy import array 
from pickle import dump 
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import LSTM 


**Loading the filename**

**We need to observe one thing that here we are loading the sequence file which we created from war and peace dataset.**

**The name of that file is char sequences**

In [ ]:
def load_doc(filename): 
# open the file as read only 
 file = open(filename, 'r') 
# read all text 
 text = file.read() 
# close the file 
 file.close() 
 return text 


**Building LSTM MODEL**

In [ ]:
def define_model(X): 
 model = Sequential() 
 model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2]))) 
 model.add(Dense(vocab_size, activation='softmax')) 
# compile model 
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # summarize defined model 
 model.summary() 
 plot_model(model, to_file='model.png', show_shapes=True) 
 return model 


**Here we will encode the sequences using one hot encoding**

In [ ]:
in_filename = 'char_sequences.txt' 
raw_text = load_doc(in_filename) 
lines = raw_text.split('\n') 
# integer encode sequences of characters 
chars = sorted(list(set(raw_text))) 
mapping = dict((c, i) for i, c in enumerate(chars)) 
sequences = list() 
for line in lines: 
# integer encode line 
 encoded_seq = [mapping[char] for char in line] 
# store 
 sequences.append(encoded_seq) 
# vocabulary size 
vocab_size = len(mapping) 
print('Vocabulary Size: %d' % vocab_size) 
# separate into input and output 
sequences = array(sequences) 
X, y = sequences[:,:-1], sequences[:,-1] 
sequences = [to_categorical(x, num_classes=vocab_size) for x in X] 
X = array(sequences)


Vocabulary Size: 93


**Here we will encode the target variable and will finally fit our model**

**The dataset provided is huge and our memory was crashing so I took 10 epochs**

**However by increasing the number of epochs our accuracy would increase**

**With 10 epochs accuracy was around 58%**



In [ ]:
y = to_categorical(y, num_classes=vocab_size) 
# define model 
model = define_model(X) 
# fit model 
model.fit(X, y, epochs=10, verbose=2) 
# save the model to file 
model.save('model.h5') 
# save the mapping 
dump(mapping, open('mapping.pkl', 'wb')) 


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 75)                50700     
                                                                 
 dense (Dense)               (None, 93)                7068      
                                                                 
Total params: 57,768
Trainable params: 57,768
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
8444/8444 - 66s - loss: 2.2927 - accuracy: 0.3562 - 66s/epoch - 8ms/step
Epoch 2/10
8444/8444 - 60s - loss: 1.8731 - accuracy: 0.4579 - 60s/epoch - 7ms/step
Epoch 3/10
8444/8444 - 60s - loss: 1.7170 - accuracy: 0.4994 - 60s/epoch - 7ms/step
Epoch 4/10
8444/8444 - 60s - loss: 1.6249 - accuracy: 0.5234 - 60s/epoch - 7ms/step
Epoch 5/10
8444/8444 - 60s - loss: 1.5640 - accuracy: 0.5398 - 60s/epoch - 7ms/step
Epoch 6/10
8444/8444 - 59s -

**We will use the learned language model to generate new sequences of text that have the same statistical properties.**

**so we will import all the important libraries**


In [ ]:
from pickle import load 
from numpy import array 
from keras.models import load_model 
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences 



In [ ]:
import numpy as np

**First we will generate characters**



**We must provide sequences of 10 characters as input to the model in order to start the generation process. We will pick these manually. A given input sequence will need to be prepared in the same way as preparing the training data for the model. First, the sequence of characters must be integer encoded using the loaded mapping**


**we will also generate the text**

In [ ]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars): 
 in_text = seed_text 
 for _ in range(n_chars): 
   # encode the characters as integers 
   encoded = [mapping[char] for char in in_text] 
   # truncate sequences to a fixed length 
   encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre') # one hot encode 
   encoded = to_categorical(encoded, num_classes=len(mapping)) 
   # predict character 
   
   predict_x=model.predict(encoded) 
   yhat=np.argmax(predict_x,axis=1)
   # reverse map integer to character 
   out_char = '' 
   for char, index in mapping.items(): 
     if index == yhat: 
       out_char = char 
       break 
   in_text += out_char 
 return in_text 
model = load_model('model.h5') 
# load the mapping 
mapping = load(open('mapping.pkl', 'rb')) 
# test start of rhyme 
print(generate_seq(model, mapping, 10, 'Sing a son', 20)) 
# test mid-line
print(generate_seq(model, mapping, 10, 'king was i', 20)) 
# test not in original 
print(generate_seq(model, mapping, 10, 'hello worl', 20)) 









Sing a son of the princess and
king was in the same to the co
hello world to the countess an


**Now we will do framing of language modelling**


**Here instead of taking the whole text file we are taking the small portion of text which is actually a nursery rhyme jack and jill went up the hill**

**so for that we created 3 models
MODEL1-One-Word-In, One-Word-Out Sequences 
MODEL2-Line-by-Line Sequence 
MODEL3-Two-Words-In, One-Word-Out Sequence**


**MODEL1:One-Word-In, One-Word-Out Sequences**

**We can start with a very simple model. Given one word as input, the model will learn to predict the next word in the sequence**

In [ ]:
from numpy import array 
from keras.preprocessing.text import Tokenizer 
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import LSTM 
from keras.layers import Embedding 


**Generating the sequence**

In [ ]:
def generate_seq(model, tokenizer, seed_text, n_words): 
  in_text, result = seed_text, seed_text 
  # generate a fixed number of words 
  for _ in range(n_words): 
# encode the text as integer 
   encoded = tokenizer.texts_to_sequences([in_text])[0] 
   encoded = array(encoded) 
   predict_x=model.predict(encoded) 
   yhat=np.argmax(predict_x,axis=1)
# map predicted word index to word 
   out_word = '' 
   for word, index in tokenizer.word_index.items(): 
    if index == yhat: 
     out_word = word 
     break 
  in_text, result = out_word, result + ' ' + out_word
  return result 








**Building the sequential model**

In [ ]:
def define_model(vocab_size): 
 model = Sequential() 
 model.add(Embedding(vocab_size, 10, input_length=1)) 
 model.add(LSTM(50)) 
 model.add(Dense(vocab_size, activation='softmax'))
 # compile network 
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # summarize defined model 
 model.summary() 
 plot_model(model, to_file='model.png', show_shapes=True) 
 return model 



**providing the source text**

**Here as the text is small so I have taken 500 epochs**

**The accuracy we got is 87%**

In [ ]:
# source text 
data = """ Jack and Jill went up the hill\n 
To fetch a pail of water\n 
Jack fell down and broke his crown\n 
And Jill came tumbling after\n """ 
# integer encode text 
tokenizer = Tokenizer() 
tokenizer.fit_on_texts([data]) 
encoded = tokenizer.texts_to_sequences([data])[0] 
# determine the vocabulary size 
vocab_size = len(tokenizer.word_index) + 1 
print('Vocabulary Size: %d' % vocab_size) 
# create word -> word sequences 
sequences = list() 
for i in range(1, len(encoded)): 
 sequence = encoded[i-1:i+1] 
 sequences.append(sequence) 
print('Total Sequences: %d' % len(sequences)) 
# split into X and y elements 
sequences = array(sequences) 
X, y = sequences[:,0],sequences[:,1] 
# one hot encode outputs 
y = to_categorical(y, num_classes=vocab_size) 
# define model 
model = define_model(vocab_size) 
# fit network 
model.fit(X, y, epochs=500, verbose=2) 
# evaluate 
print(generate_seq(model, tokenizer, 'Jack', 6)) 
 


Vocabulary Size: 22
Total Sequences: 24
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             220       
                                                                 
 lstm_1 (LSTM)               (None, 50)                12200     
                                                                 
 dense_1 (Dense)             (None, 22)                1122      
                                                                 
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
1/1 - 2s - loss: 3.0903 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 2/500
1/1 - 0s - loss: 3.0895 - accuracy: 0.0833 - 9ms/epoch - 9ms/step
Epoch 3/500
1/1 - 0s - loss: 3.0887 - accuracy: 0.1250 - 8ms/epoch - 8ms/step
Epoch 4/500
1/1 - 0s - loss: 3.0879 - 

**MODEL2: Line-by-Line Sequence**

**Here we will split up the source text line-by-line, then break each line down into a series of words that build up** 


In [ ]:

from numpy import array 
from keras.preprocessing.text import Tokenizer 
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences 
from keras.utils.vis_utils import plot_model 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import LSTM 
from keras.layers import Embedding 


**Generating the sequence**

**defining the model**

**accuracy was around 96%**

In [ ]:
def generate_seq(model, tokenizer, max_length, seed_text, n_words): 
 in_text = seed_text 
# generate a fixed number of words 
 for _ in range(n_words): 
# encode the text as integer 
  encoded = tokenizer.texts_to_sequences([in_text])[0] 
# pre-pad sequences to a fixed length 
  encoded = pad_sequences([encoded], maxlen=max_length, padding='pre') 
# predict probabilities for each word 
  predict_x=model.predict(encoded) 
  yhat=np.argmax(predict_x,axis=1)
# map predicted word index to word 
  out_word = '' 
  for word, index in tokenizer.word_index.items(): 
    if index == yhat: 
      out_word = word 
      break 
# append to input 
  in_text += ' ' + out_word 
 return in_text 
# define the model 
def define_model(vocab_size, max_length): 
 model = Sequential() 
 model.add(Embedding(vocab_size, 10, input_length=max_length-1)) 
 model.add(LSTM(50)) 
 model.add(Dense(vocab_size, activation='softmax')) 
# compile network 
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # summarize defined model 
 model.summary() 
 plot_model(model, to_file='model.png', show_shapes=True) 
 return model 
# source text 
data = """ Jack and Jill went up the hill\n 
To fetch a pail of water\n
19.5. Model 2: Line-by-Line Sequence 220 
Jack fell down and broke his crown\n 
And Jill came tumbling after\n """ 
# prepare the tokenizer on the source text 
tokenizer = Tokenizer() 
tokenizer.fit_on_texts([data]) 
# determine the vocabulary size 
vocab_size = len(tokenizer.word_index) + 1 
print('Vocabulary Size: %d' % vocab_size) 
# create line-based sequences 
sequences = list() 
for line in data.split('\n'): 
 encoded = tokenizer.texts_to_sequences([line])[0] 
 for i in range(1, len(encoded)): 
  sequence = encoded[:i+1] 
  sequences.append(sequence) 
print('Total Sequences: %d' % len(sequences)) 
# pad input sequences 
max_length = max([len(seq) for seq in sequences]) 
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre') 
print('Max Sequence Length: %d' % max_length) 
# split into input and output elements 
sequences = array(sequences) 
X, y = sequences[:,:-1],sequences[:,-1] 
y = to_categorical(y, num_classes=vocab_size) 
# define model 
model = define_model(vocab_size, max_length) 
# fit network 
model.fit(X, y, epochs=500, verbose=2) 
# evaluate model 
print(generate_seq(model, tokenizer, max_length-1, 'Jack', 4)) 
print(generate_seq(model, tokenizer, max_length-1, 'Jill', 4)) 


Vocabulary Size: 30
Total Sequences: 29
Max Sequence Length: 9
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 8, 10)             300       
                                                                 
 lstm_2 (LSTM)               (None, 50)                12200     
                                                                 
 dense_2 (Dense)             (None, 30)                1530      
                                                                 
Total params: 14,030
Trainable params: 14,030
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
1/1 - 2s - loss: 3.4019 - accuracy: 0.0345 - 2s/epoch - 2s/step
Epoch 2/500
1/1 - 0s - loss: 3.4004 - accuracy: 0.0345 - 15ms/epoch - 15ms/step
Epoch 3/500
1/1 - 0s - loss: 3.3990 - accuracy: 0.0690 - 11ms/epoch - 11ms/step
Epoch 4/500
1/1

**MODEL 3:Two-Words-In, One-Word-Out Sequence**

**We can use an intermediate between the one-word-in and the whole-sentence-in approaches and pass in a sub-sequences of words as input. This will provide a trade-off between the two framings allowing new lines to be generated and for generation to be picked up mid line**




In [ ]:
from numpy import array 
from keras.preprocessing.text import Tokenizer 
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences 
from keras.utils.vis_utils import plot_model 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import LSTM 
from keras.layers import Embedding 


**Creating sequences,**
**Building the Sequential model and**
**Accuracy was around 95%**

In [ ]:
# generate a sequence from a language model 
def generate_seq(model, tokenizer, max_length, seed_text, n_words): 
 in_text = seed_text 
# generate a fixed number of words 
 for _ in range(n_words): 
# encode the text as integer 
  encoded = tokenizer.texts_to_sequences([in_text])[0] 
# pre-pad sequences to a fixed length 
  encoded = pad_sequences([encoded], maxlen=max_length, padding='pre') 
# predict probabilities for each word 
  predict_x=model.predict(encoded) 
  yhat=np.argmax(predict_x,axis=1)
# map predicted word index to word 
  out_word = ''
  for word, index in tokenizer.word_index.items(): 
    if index == yhat: 
      out_word = word 
      break 
# append to input 
  in_text += ' ' + out_word 
 return in_text 
# define the model 
def define_model(vocab_size, max_length): 
 model = Sequential() 
 model.add(Embedding(vocab_size, 10, input_length=max_length-1)) 
 model.add(LSTM(50)) 
 model.add(Dense(vocab_size, activation='softmax')) 
# compile network 
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # summarize defined model 
 model.summary() 
 plot_model(model, to_file='model.png', show_shapes=True) 
 return model 
# source text 
data = """ Jack and Jill went up the hill\n 
To fetch a pail of water\n 
Jack fell down and broke his crown\n 
And Jill came tumbling after\n """ 
# integer encode sequences of words 
tokenizer = Tokenizer() 
tokenizer.fit_on_texts([data]) 
encoded = tokenizer.texts_to_sequences([data])[0] 
# retrieve vocabulary size 
vocab_size = len(tokenizer.word_index) + 1 
print('Vocabulary Size: %d' % vocab_size) 
# encode 2 words -> 1 word 
sequences = list() 
for i in range(2, len(encoded)): 
  sequence = encoded[i-2:i+1] 
  sequences.append(sequence) 
print('Total Sequences: %d' % len(sequences)) 
# pad sequences 
max_length = max([len(seq) for seq in sequences]) 
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length) 
# split into input and output elements 
sequences = array(sequences) 
X, y = sequences[:,:-1],sequences[:,-1] 
y = to_categorical(y, num_classes=vocab_size) 
# define model 
model = define_model(vocab_size, max_length) 
# fit network 
model.fit(X, y, epochs=500, verbose=2) 
# evaluate model 
print(generate_seq(model, tokenizer, max_length-1, 'Jack and', 5)) 
print(generate_seq(model, tokenizer, max_length-1, 'And Jill', 3)) 
print(generate_seq(model, tokenizer, max_length-1, 'fell down', 5)) 
print(generate_seq(model, tokenizer, max_length-1, 'pail of', 5))


Vocabulary Size: 22
Total Sequences: 23
Max Sequence Length: 3
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 2, 10)             220       
                                                                 
 lstm_3 (LSTM)               (None, 50)                12200     
                                                                 
 dense_3 (Dense)             (None, 22)                1122      
                                                                 
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
1/1 - 4s - loss: 3.0899 - accuracy: 0.0435 - 4s/epoch - 4s/step
Epoch 2/500
1/1 - 0s - loss: 3.0890 - accuracy: 0.1304 - 10ms/epoch - 10ms/step
Epoch 3/500
1/1 - 0s - loss: 3.0880 - accuracy: 0.0870 - 12ms/epoch - 12ms/step
Epoch 4/500
1/1

**NOW here we will develop a Neural Language Model for Text Generation**

**On the basis of whatever is taught in the above examples**

**The dataset we will be taking will be of war and peace dataset which is actually a very large text file** 


**SO we will start the process by:**

**DATA PREPARATION which requires cleaning of data such as removing the whitespaces and punctuations**

**GENERATE SEQUENCES means we will generate character sequences from the clean data**

**DEFINING THE MODEL means we will be implementing the sequential model**




In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**SO here we are cleaning the text file by removing punctuations and whitespaces**

**we will convert each character into tokens**'

**we will create sequences from that tokens**



In [ ]:
import string 
import re 
# load doc into memory 
def load_doc(filename): 
# open the file as read only 
 file = open(filename, 'r') 
# read all text 
 text = file.read() 
# close the file 
 file.close() 
 return text 
# turn a doc into clean tokens 
def clean_doc(doc): 
# replace '--' with a space ' ' 
 doc = doc.replace('--', ' ') 
# split into tokens by white space 
 tokens = doc.split() 
# prepare regex for char filtering 
 re_punc = re.compile('[%s]' % re.escape(string.punctuation)) 
# remove punctuation from each word 
 tokens = [re_punc.sub('', w) for w in tokens] 
# remove remaining tokens that are not alphabetic 
 tokens = [word for word in tokens if word.isalpha()] 
# make lower case 
 tokens = [word.lower() for word in tokens] 
 return tokens 
def save_doc(lines, filename): 
 data = '\n'.join(lines) 
 file = open(filename, 'w') 
 file.write(data) 
 file.close() 
# load document 
in_filename = '/content/drive/My Drive/war_and_peace.txt' 
doc = load_doc(in_filename) 
print(doc[:200]) 
# clean document 
tokens = clean_doc(doc) 
print(tokens[:200]) 
print('Total Tokens: %d' % len(tokens)) 
print('Unique Tokens: %d' % len(set(tokens))) 
# organize into sequences of tokens 
length = 50 + 1 
sequences = list() 
for i in range(length, len(tokens)): 
# select sequence of tokens 
 seq = tokens[i-length:i] 
# convert into a line 
 line = ' '.join(seq) 
# store 
 sequences.append(line) 
print('Total Sequences: %d' % len(sequences)) 
# save sequences to file 
out_filename = 'war_and_peace.txt' 
save_doc(sequences, out_filename) 


CHAPTER I

“Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don’t tell me that this means war,
if you still try to defend the infamies and horro
['chapter', 'i', 'prince', 'so', 'genoa', 'and', 'lucca', 'are', 'now', 'just', 'family', 'estates', 'of', 'the', 'buonapartes', 'but', 'i', 'warn', 'you', 'if', 'you', 'tell', 'me', 'that', 'this', 'means', 'war', 'if', 'you', 'still', 'try', 'to', 'defend', 'the', 'infamies', 'and', 'horrors', 'perpetrated', 'by', 'that', 'really', 'believe', 'he', 'is', 'will', 'have', 'nothing', 'more', 'to', 'do', 'with', 'you', 'and', 'you', 'are', 'no', 'longer', 'my', 'friend', 'no', 'longer', 'my', 'as', 'you', 'call', 'yourself', 'but', 'how', 'do', 'you', 'do', 'i', 'see', 'i', 'have', 'frightened', 'down', 'and', 'tell', 'me', 'all', 'the', 'it', 'was', 'in', 'july', 'and', 'the', 'speaker', 'was', 'the', 'wellknown', 'anna', 'pávlovna', 'schérer', 'maid', 'of', 'honor', 'and', 'favorite', 'of

In [ ]:
from numpy import array 
from pickle import dump 
from keras.preprocessing.text import Tokenizer 
from keras.utils.vis_utils import plot_model 
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential 


**So here we will be importing all the libraries required to Build our model**

In [ ]:
from keras.layers import Dense 
from keras.layers import LSTM 
from keras.layers import Embedding 


**Since the dataset is very large we are taking 10 epochs only**


**By increasing epochs accuracy can be increased**

**Here the accuracy was around 12% because of the smaller epochs**


In [ ]:
# load doc into memory 
def load_doc(filename): 
# open the file as read only 
 file = open(filename, 'r') 
# read all text 
 text = file.read() 
# close the file 
 file.close() 
 return text 
# define the model 
def define_model(vocab_size, seq_length): 
 model = Sequential() 
 model.add(Embedding(vocab_size, 50, input_length=seq_length)) 
 model.add(LSTM(100, return_sequences=True)) 
 model.add(LSTM(100)) 
 model.add(Dense(100, activation='relu'))
 model.add(Dense(vocab_size, activation='softmax')) 
# compile network 
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # summarize defined model 
 model.summary() 
 plot_model(model, to_file='model.png', show_shapes=True) 
 return model 
# load 
in_filename = 'war_and_peace.txt' 
doc = load_doc(in_filename) 
lines = doc.split('\n') 
# integer encode sequences of words 
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(lines) 
sequences = tokenizer.texts_to_sequences(lines) 
# vocabulary size 
vocab_size = len(tokenizer.word_index) + 1 
# separate into input and output 
sequences = array(sequences) 
X, y = sequences[:,:-1], sequences[:,-1] 
y = to_categorical(y, num_classes=vocab_size) 
seq_length = X.shape[1] 
# define model 
model = define_model(vocab_size, seq_length) 
# fit model 
model.fit(X, y, batch_size=128, epochs=10) 
# save the model to file 
model.save('model.h5') 
# save the tokenizer 
dump(tokenizer, open('tokenizer.pkl', 'wb')) 


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 50, 50)            248450    
                                                                 
 lstm_8 (LSTM)               (None, 50, 100)           60400     
                                                                 
 lstm_9 (LSTM)               (None, 100)               80400     
                                                                 
 dense_8 (Dense)             (None, 100)               10100     
                                                                 
 dense_9 (Dense)             (None, 4969)              501869    
                                                                 
Total params: 901,219
Trainable params: 901,219
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
352/352 [============================

**Now that we have a trained language model, we can use it. In this case, we can use it to generate new sequences of text that have the same statistical properties as the source text**



In [ ]:
from random import randint 
from pickle import load 
from keras.models import load_model 
from keras.preprocessing.sequence import pad_sequences 


**So, Here as we now have a trained language model, we can use it. In this case, we can use it to generate new sequences of text that have the same statistical properties as the source text** 

In [ ]:
def load_doc(filename): 
# open the file as read only 
 file = open(filename, 'r') 
# read all text 
 text = file.read() 
# close the file 
 file.close() 
 return text 
# generate a sequence from a language model 
def generate_seq(model, tokenizer, seq_length, seed_text, n_words): 
 result = list() 
 in_text = seed_text
 # generate a fixed number of words 
 for _ in range(n_words):  
   # encode the text as integer 
  encoded = tokenizer.texts_to_sequences([in_text])[0] 
# truncate sequences to a fixed length 
  encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre') # predict probabilities for each word 
  predict_x=model.predict(encoded) 
  yhat=np.argmax(predict_x,axis=1)
# map predicted word index to word 
  out_word = '' 
  for word, index in tokenizer.word_index.items(): 
   if index == yhat: 
    out_word = word 
    break 
# append to input 
  in_text += ' ' + out_word 
  result.append(out_word) 
 return ' '.join(result) 
# load cleaned text sequences 
in_filename = 'war_and_peace.txt' 
doc = load_doc(in_filename) 
lines = doc.split('\n') 
seq_length = len(lines[0].split()) - 1 
# load the model 
model = load_model('model.h5') 
# load the tokenizer 
tokenizer = load(open('tokenizer.pkl', 'rb'))
# select a seed text 
seed_text = lines[randint(0,len(lines))] 
print(seed_text + '\n') 
# generate new text 
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50) 
print(generated) 




down beside dr lorrain who was sitting in a graceful pose under a portrait of catherine leaning his elbow on a table said the doctor in answer to a remark about the weather weather is beautiful princess and besides in moscow one feels as if one were in the replied the

princess and the man and the man and the man and the man and the man and the man and the man and the man and the man and the man and the man and the man and the man and the man and the man and the man and
